# Watson ML for Developers

## Import, Clean, and Analyze Data
We are going to use PixieDust to load and visualize our data.

In [1]:
import pixiedust

Pixiedust database opened successfully


In [2]:
df = pixiedust.sampleData("https://raw.githubusercontent.com/markwatsonatx/watson-ml-for-developers/master/data/house-prices.csv")

Downloaded 92 bytes
Creating pySpark DataFrame for 'https://raw.githubusercontent.com/markwatsonatx/watson-ml-for-developers/master/data/house-prices.csv'. Please wait...
Loading file using 'SparkSession'
Successfully created pySpark DataFrame for 'https://raw.githubusercontent.com/markwatsonatx/watson-ml-for-developers/master/data/house-prices.csv'


In [3]:
display(df)

SquareFeet,Bedrooms,Color,Price
2100,3,White,100000
2300,4,White,125000
2500,4,Brown,150000


## Build a Machine Learning Model with Spark ML 

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [5]:
assembler = VectorAssembler(inputCols=['SquareFeet','Bedrooms'],outputCol="features")
lr = LinearRegression(labelCol='Price', featuresCol='features')
pipeline = Pipeline(stages=[assembler, lr])
model = pipeline.fit(df)

# Deploy Model to Watson ML

In [6]:
import json
import requests
import urllib3

### Fill in Your Watson ML Credentials

In [7]:
service_path = 'https://eu-gb.ml.cloud.ibm.com'
username = 'd5cc912e-6a8e-48a8-955a-f6e00fbcc6df'
password = '365af552-9cf4-44d0-b08f-5b5c997d2124'
instance_id = 'd6ef2837-dcb0-4670-a1cf-0acee4ab7d93'
model_name = 'House Prices Model'
deployment_name = 'House Prices Deployment'

In [8]:
# The code was removed by Watson Studio for sharing.

### Initialize Watson ML Repository Libraries

In [9]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact
ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

### Save Model to Watson ML Repository

In [10]:
pipeline_artifact = MLRepositoryArtifact(pipeline, name="pipeline")
model_artifact = MLRepositoryArtifact(model, training_data=df, name=model_name, pipeline_artifact=pipeline_artifact)
saved_model = ml_repository_client.models.save(model_artifact)
model_id = saved_model.uid

In [11]:
headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
ml_token = 'Bearer ' + json.loads(response.text).get('token')

### Create a Deployment

In [12]:
deployment_url = service_path + "/v3/wml_instances/" + instance_id + "/published_models/" + model_id + "/deployments/"
deployment_header = {'Content-Type': 'application/json', 'Authorization': ml_token}
deployment_payload = {"type": "online", "name": deployment_name}
deployment_response = requests.post(deployment_url, json=deployment_payload, headers=deployment_header)
deployment_response.text
scoring_url = json.loads(deployment_response.text).get('entity').get('scoring_url')
scoring_url

u'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/d6ef2837-dcb0-4670-a1cf-0acee4ab7d93/published_models/06ed917d-c132-4514-bb60-2da090f7a248/deployments/d8fa0d22-e5c4-4f4e-8864-5f5265a4f504/online'

### Test the Deployment

In [13]:
def get_prediction_from_watson_ml(square_feet, num_bedrooms):
    scoring_header = {'Content-Type': 'application/json', 'Authorization': ml_token}
    scoring_payload = {'fields': ['SquareFeet','Bedrooms'], 'values': [[square_feet, num_bedrooms]]}
    scoring_response = requests.post(scoring_url, json=scoring_payload, headers=scoring_header)
    return scoring_response.text

In [14]:
response = get_prediction_from_watson_ml(2400, 4)
print response

{
  "fields": ["SquareFeet", "Bedrooms", "features", "prediction"],
  "values": [[2400, 4, [2400.0, 4.0], 137499.99999999968]]
}
